In [ ]:
#installing openCV 
%pip install opencv-python

In [ ]:
#import libraries
import numpy as np
import cv2

# Task 1

In [ ]:
#loading video object with video file
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

#check if video is opened successfully
if(video.isOpened() == False):
    print("Error opening video file!")

#threshold area
threshold_area = 3000

#marking out main street to focus detection on
mainStreet_x, mainStreet_y, mainStreet_w, mainStreet_h = 0, 350, 1024, 250

    
#background subtraction
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

while True:
    #read frames
    check, frame = video.read()
    
    #if false leave loop
    if not check:
        break
        
    #detect cars ONLY from main street
    main_street = frame[mainStreet_y:mainStreet_y + mainStreet_h, mainStreet_x:mainStreet_x + mainStreet_w]
    
    #grayscale
    gray_scale = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
    
    #smoothen image and reduce noise
    blur_frame = cv2.GaussianBlur(gray_scale, (25, 25), 0)
    
    #frame difference
    frame_diff =cv2.absdiff(gray_scale, blur_frame)
    
    #background subtraction
    fg_mask = bg_subtractor.apply(frame_diff)
    
    #convert to binary image
    _, binary_mask = cv2.threshold(fg_mask, 50, 255, cv2.THRESH_BINARY)
    
    #find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #each contour draw rectangle around cars
    for c in contours:
        #cordinates of contour box
        (x, y, w, h) = cv2.boundingRect(c)
        
        #filter out any contours less than threshold area
        if cv2.contourArea(c) < threshold_area: 
            continue
        
        #draw rectangle box around contour
        cv2.rectangle(main_street, (x, y), (x+w, y+h), (0, 255, 0), 1)
    
    #display video feed with traffic detection
    cv2.imshow("Traffic detection", frame)
    
    #exit feed window with "q" key pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release video capture object and destroy windows
video.release()
cv2.destroyAllWindows()

# Task 2

In [ ]:
#class to track contour box movement
class Car_contour:
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.direction = ""
        self.counted = False

## Video 1

In [ ]:
from datetime import datetime

#Traffic_Laramie_1.mp4
video_1 = cv2.VideoCapture("Traffic_Laramie_1.mp4")

#checking for error opening video file
if (video_1.isOpened() == False):
    print("Error opening video 1!")
    
#threshold area
threshold_area = 3000

#counter for number of cars
counter_cars = 0

#average number of cars
avg_cars = 0

#marking out main street to focus detection on
mainStreet_x, mainStreet_y, mainStreet_w, mainStreet_h = 0, 250, 1024, 350

#background subtraction
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

#store cars instance
previous_car = []

#counter to track car in counter zone
counter = 0

#start time
start_time = datetime.now()

while True:
    #store cars instance
    current_car = []
   
    #read frames from each video file
    check_1, frame_1 = video_1.read()
    
    #if false leave loop
    if not check_1:
        break
    
    #detect cars ONLY from main street
    main_street = frame_1[mainStreet_y:mainStreet_y + mainStreet_h, mainStreet_x:mainStreet_x + mainStreet_w]
    
    #cordinates for counter box in red
    point_x_counter = 300
    point_y_counter = 60
    
    #cordinates for detection box in green
    point_x_detection = 160
    point_y_detection = 60
    
    #draw counter box
    cv2.rectangle(main_street, (point_x_counter, point_y_counter), (point_x_counter + 30, point_y_counter + 80), (0, 0, 255), 2)
    
    #draw detection box
    cv2.rectangle(main_street, (point_x_detection, point_y_detection), (point_x_detection + 30, point_y_detection + 80), (0, 255, 255), 2)
    
    #grayscale
    gray_scale = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
    
    #smoothen image and reduce noise
    blur_frame = cv2.GaussianBlur(gray_scale, (25, 25), 0)
    
    #frame difference
    frame_diff =cv2.absdiff(gray_scale, blur_frame)
    
    #background subtraction
    fg_mask = bg_subtractor.apply(frame_diff)
    
    #convert to binary image
    _, binary_mask = cv2.threshold(fg_mask, 50, 255, cv2.THRESH_BINARY)
    
    #find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #each contour draw rectangle around cars
    for c in contours:
        #cordinates of contour box
        (x, y, w, h) = cv2.boundingRect(c)
        
        
        #filter out any contours less than threshold area
        if cv2.contourArea(c) < threshold_area: 
            continue
        
        
        #drawing rectangle around contour
        cv2.rectangle(main_street, (x, y), (x+w, y+h), (0, 255, 0), 1)
            
        current_car.append(Car_contour(x,y,w,h))
    
    
    for curr in current_car:
        
        for prev in previous_car:
            #calculating the difference in the x value
            dist_x = curr.x - prev.x
            
            #determine direction
            if dist_x < 0:
                curr.direction = "left"
            elif dist_x > 0:
                curr.direction = "right"
        
        #checking if it passes counter box to start counter
        if curr.x < point_x_counter + 30 and curr.x > point_x_counter and curr.y < point_y_counter + 80 and curr.direction == "left":
            counter += 1

        #checking if car has entered detection box
        if curr.x < point_x_detection + 30 and curr.x > point_x_detection and curr. y < point_y_detection + 80 and curr.direction == "left":  
            if counter > 0:
                counter_cars += 1
                counter = 0
                
           
        
    #updating the previous car array
    previous_car = current_car
    
    #calculating average cars per min
    current_time = datetime.now()
    time_diff = current_time - start_time
    time_diff = time_diff.total_seconds()
    avg_cars = (counter_cars/ time_diff) * 60
    
    #rounding to 2dp
    avg_cars = round(avg_cars, 2)
    
    #text
    cv2.putText(frame_1, f'Car count: {counter_cars}, {avg_cars} cars/min', (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    #display video feed
    cv2.imshow("Traffic_Laramie_1", frame_1)
    
    #exit feed window with "q" key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#printing number of cars
print("Traffic_Laramie_1.mp4\n")
print(f"Number of cars entering city centre: {counter_cars}")
print(f"Cars per min: {avg_cars} cars/min")
        
#release video capture object and destroy windows
video_1.release()
cv2.destroyAllWindows()

## Video 2

In [ ]:
#Traffic_Laramie_2.mp4
video_2 = cv2.VideoCapture("Traffic_Laramie_2.mp4")

#checking for error opening video file
if (video_2.isOpened() == False):
    print("Error opening video 2!")
    
#threshold area
threshold_area = 3000

#counter for number of cars
counter_cars = 0

#average number of cars
avg_cars = 0

#marking out main street to focus detection on
mainStreet_x, mainStreet_y, mainStreet_w, mainStreet_h = 0, 250, 1024, 350

#background subtraction
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

#store cars instance
previous_car = []

#counter to track car in counter zone
counter = 0

#start time
start_time = datetime.now()

while True:
    #store cars instance
    current_car = []
   
    #read frames from each video file
    check_2, frame_2 = video_2.read()
    
    #if false leave loop
    if not check_2:
        break
    
    #detect cars ONLY from main street
    main_street = frame_2[mainStreet_y:mainStreet_y + mainStreet_h, mainStreet_x:mainStreet_x + mainStreet_w]
    
    #cordinates for counter box in red
    point_x_counter = 300
    point_y_counter = 60
    
    #cordinates for detection box in green
    point_x_detection = 150
    point_y_detection = 60
    
    #draw counter box
    cv2.rectangle(main_street, (point_x_counter, point_y_counter), (point_x_counter + 30, point_y_counter + 80), (0, 0, 255), 2)
    
    #draw detection box
    cv2.rectangle(main_street, (point_x_detection, point_y_detection), (point_x_detection + 30, point_y_detection + 80), (0, 255, 255), 2)
    
    #grayscale
    gray_scale = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
    
    #smoothen image and reduce noise
    blur_frame = cv2.GaussianBlur(gray_scale, (25, 25), 0)
    
    #frame difference
    frame_diff =cv2.absdiff(gray_scale, blur_frame)
    
    #background subtraction
    fg_mask = bg_subtractor.apply(frame_diff)
    
    #convert to binary image
    _, binary_mask = cv2.threshold(fg_mask, 50, 255, cv2.THRESH_BINARY)
    
    #find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #each contour draw rectangle around cars
    for c in contours:
        #cordinates of contour box
        (x, y, w, h) = cv2.boundingRect(c)
        
        
        #filter out any contours less than threshold area
        if cv2.contourArea(c) < threshold_area: 
            continue
        
        
        #drawing rectangle around contour
        cv2.rectangle(main_street, (x, y), (x+w, y+h), (0, 255, 0), 1)
            
        current_car.append(Car_contour(x,y,w,h))
    
    
    for curr in current_car:
        
        for prev in previous_car:
            #calculating the difference in the x value
            dist_x = curr.x - prev.x
            
            #determine direction
            if dist_x < 0:
                curr.direction = "left"
            elif dist_x > 0:
                curr.direction = "right"
        
        #checking if it passes counter box to start counter
        if curr.x < point_x_counter + 30 and curr.x > point_x_counter and curr.y < point_y_counter + 80 and curr.direction == "left":
            counter += 1
            
            
        #checking if car has entered detection box
        if curr.x < point_x_detection + 30 and curr.x > point_x_detection and curr. y < point_y_detection + 80 and curr.direction == "left":  
            if counter > 0:
                counter_cars += 1
                counter = 0
           
        
    #updating the previous car array
    previous_car = current_car
    
    #calculating average cars per min
    current_time = datetime.now()
    time_diff = current_time - start_time
    time_diff = time_diff.total_seconds()
    avg_cars = (counter_cars / time_diff) * 60
    
    #rounding to 2dp
    avg_cars = round(avg_cars, 2)
    
    #text
    cv2.putText(frame_2, f'Car count: {counter_cars}, {avg_cars} cars/min', (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    #display video feed
    cv2.imshow("Traffic_Laramie_2", frame_2)
    
    #exit feed window with "q" key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
#printing number of cars
print("Traffic_Laramie_2.mp4\n")
print(f"Number of cars entering city centre: {counter_cars}")
print(f"Cars per min: {avg_cars} cars/min")


#release video capture object and destroy windows
video_2.release()
cv2.destroyAllWindows()